# 1 Introduction to Semantic Kernel

Semantic Kernel is an open-source framework designed to seamlessly integrate AI models into applications. It provides a unified approach to managing AI capabilities, enabling developers to build intelligent, context-aware systems efficiently.

---

### 1.1 Key Concepts
- **Kernel**: The core component that orchestrates AI services, plugins, and agents.
- **Plugins**: Modular units that encapsulate specific functionalities, such as interacting with APIs or performing computations.
- **Agents**: AI-driven entities that use the kernel to process inputs, maintain context, and generate responses.
- **Chat Completion**: A feature that allows conversational interactions with AI models, maintaining context across exchanges.
- **Planner**: A mechanism to break down complex tasks into smaller, manageable steps that can be executed sequentially or in parallel.
- **Filters**: Tools to refine and customize the output of AI models based on specific criteria or constraints.
- **Vector Store (Memory) Connectors**: Integrations that allow the kernel to store and retrieve embeddings for long-term memory, enabling context retention across sessions.
- **Prompt Templates**: Predefined templates that structure the input prompts for AI models, ensuring consistency and efficiency in generating responses.

---

### 1.2 Components
1. **AI Services**: Integrates with OpenAI, Azure OpenAI, and other AI providers for tasks like text generation and summarization.
2. **Plugins**: Extensible modules for tasks like GitHub integration, data analysis, or custom workflows.
3. **Execution Settings**: Configurations for managing AI model behavior, such as token limits and response formats.
4. **Chat History**: Maintains a record of interactions to provide context-aware responses.
5. **Planner**: Enables dynamic task planning and execution by breaking down user requests into actionable steps.
6. **Vector Store (Memory)**: Stores embeddings for long-term memory, allowing the kernel to recall past interactions or knowledge.
7. **Prompt Templates**: Simplifies the creation of structured prompts for consistent and effective communication with AI models.

---

### 1.3 Real-World Examples
- **Customer Support**: Automating responses to common queries while escalating complex issues to human agents.
- **Code Review**: Analyzing pull requests and suggesting improvements using GitHub plugins.
- **Content Creation**: Generating blog posts, summaries, or reports with minimal input.
- **Data Analysis**: Extracting insights from datasets and presenting them in a user-friendly format.
- **Task Automation**: Using planners to decompose and execute complex workflows.
- **Contextual Search**: Leveraging vector stores to retrieve relevant information based on embeddings.

---

Semantic Kernel empowers developers to create intelligent, modular, and extensible applications by combining the power of AI with a unified framework. Its support for planners, filters, vector stores, and prompt templates makes it a versatile tool for building advanced AI-driven solutions.

# 2. Semantic Kernel Git Chat Agent

The Semantic Kernel Git Chat Agent is designed to facilitate chat-based interactions with an AI model, leveraging the power of Semantic Kernel's unified framework. This framework integrates the chat-completion capabilities of various AI models, enabling seamless communication.

The Git Chat Agent maintains a chat history that is presented to the AI model with each request, ensuring context-aware responses. It can generate responses directed to users or interact with other agents, making it a versatile tool for managing GitHub repositories and related tasks.

# 3. Prerequisites

Before proceeding with the steps in this notebook, ensure the following prerequisites are met:

---

### 1. Generate a GitHub Personal Access Token (PAT)
A Personal Access Token (PAT) is required to interact with the GitHub API. Follow these steps to generate a PAT:

1. Log in to your GitHub account.
2. Navigate to **Settings** > **Developer settings** > **Personal access tokens** > **Tokens (classic)**.
3. Click on **Generate new token**.
4. Select the required scopes:
   - **repo**: Full control of private repositories.
   - **read:org**: Read-only access to organization membership.
   - **user**: Read access to user profile data.
5. Set an expiration date for the token.
6. Click **Generate token** and copy the token. **Save it securely**, as it will not be shown again.

---

### 2. Add Keys to the `.env` File
Rename the `.env.example` to `.env` file in the root directory of your project and add the following keys:

```plaintext
# OpenAI API Keys
AZURE_OPENAI_ENDPOINT=<Your Azure OpenAI Endpoint> -- will be provided during the lab
AZURE_OPENAI_API_KEY=<Your Azure OpenAI API Key> -- will be provided during the lab

# GitHub PAT
GITHUB_PAT=<Your GitHub Personal Access Token> -- Add your PAT from above step

### Check the current version of Semantic Kernel.

In [1]:
from semantic_kernel import __version__

__version__ #1.29.0

'1.29.0'

### Load variables from .env file for local development

In [3]:
from dotenv import load_dotenv
import os

if os.path.exists(".env"):
    load_dotenv(override=True)
else:
    # Load environment variables from the parent 
    current_dir = os.getcwd()
    env_path = os.path.join(current_dir, '..', '.env')
    load_dotenv(dotenv_path=env_path)

# 3.Creating a chat completion service

>Note: The AzureChatCompletion service also supports Microsoft Entra authentication. If you don't provide an API key, the service will attempt to authenticate using the Entra token.

In [16]:
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

# Add Azure OpenAI chat completion
chat_completion = AzureChatCompletion(
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    service_id="serv-git-chat-1" # Used for calling the service in the kernel and settings
)

### 3.1 Let's try chatting with LLM model , if this works you are good to go for next steps

In [ ]:
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings
from semantic_kernel.contents.chat_history import ChatHistory

execution_settings = AzureChatPromptExecutionSettings()

system_message = """
You are a helpful assistant that can answer questions and provide information.
"""

chat_history = ChatHistory(system_message=system_message)

chat_history.add_user_message("Hello, who are you , what's your model name/version , how much token length you can accept from Azure deployment?")

response = await chat_completion.get_chat_message_content(
    chat_history=chat_history,
    settings=execution_settings,
)

print(response)

## 4. Creating a Chat Completion Agent

A chat completion agent is fundamentally based on AI services. Creating a chat completion agent starts with creating a Kernel instance that contains one or more chat-completion services and then instantiating the agent with a reference to that Kernel instance.

---

### Difference Between Chat Completion Service and Chat Completion Agent

- **Chat Completion Service**:
  - A low-level interface that directly interacts with the AI model to process and generate responses.
  - It requires explicit management of chat history, prompt formatting, and execution settings.
  - Example: AzureChatCompletion is a service that connects to Azure OpenAI to handle chat-based interactions.

- **Chat Completion Agent**:
  - A higher-level abstraction built on top of chat completion services.
  - It manages chat history, context, and execution settings automatically, simplifying the interaction process.
  - Agents can integrate additional functionalities, such as plugins or task-specific instructions, to enhance their capabilities.
  - Example: ChatCompletionAgent uses a Kernel instance with one or more services to provide intelligent, context-aware responses.

By combining chat completion services with agents, developers can build robust, context-aware conversational systems with minimal effort.

In [18]:
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.functions.kernel_arguments import KernelArguments
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings

# Define the Kernel
kernel = Kernel()

# Add the AzureChatCompletion AI Service to the Kernel
kernel.add_service(chat_completion)

settings = AzureChatPromptExecutionSettings(service_id="serv-git-chat-1")

# Create the agent
agent = ChatCompletionAgent(
    kernel=kernel, 
    name="chat-agent", 
    instructions="You are a helpful agent.",
    arguments=KernelArguments(settings=settings)
)

## 4.1 Conversing with ChatCompletionAgent

- There are multiple ways to converse with a ChatCompletionAgent.

- The easiest is to call and await get_response:

In [ ]:
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents import AuthorRole

# Define the chat history
history = ChatHistory()

# Add the user message
history.add_message(ChatMessageContent(role=AuthorRole.USER, content="Hello, how are you?"))

# Generate the agent response
response = await agent.get_response(messages=history,arguments=KernelArguments(settings=settings))

print(f"{response.content}")

## 5. Creating a Semantic Kernel Plugin

Plugins in Semantic Kernel are modular units designed to encapsulate specific functionalities, making it easier to extend the capabilities of the kernel. In this lab, we are creating a plugin to interact with the GitHub API, enabling the retrieval of repository and user information, as well as managing issues and commits.

The purpose of creating this plugin is to demonstrate how Semantic Kernel can be integrated with external APIs to build intelligent, task-specific solutions. By using plugins, we can simplify complex workflows and enhance the functionality of the chat agent.

This plugin example was provided by Microsoft at this link: [GitHub Plugin Example](https://github.com/microsoft/semantic-kernel/blob/main/python/samples/learn_resources/plugins/GithubPlugin/github.py).

---

### Alternative Approach

You can also use the **MCP Server Agents** to perform similar tasks. MCP Server Agents provide a robust framework for managing and automating tasks using pre-configured agents. For more information, refer to the following resources:

- [MCP Server Agents Documentation](https://docs.anthropic.com/en/docs/agents-and-tools/mcp)
- [MCP GitHub Repository](https://github.com/modelcontextprotocol)

In [20]:
import httpx
from pydantic import BaseModel, Field

from semantic_kernel.functions.kernel_function_decorator import kernel_function

# region GitHub Models


class Repo(BaseModel):
    id: int = Field(..., alias="id")
    name: str = Field(..., alias="full_name")
    description: str | None = Field(default=None, alias="description")
    url: str = Field(..., alias="html_url")


class User(BaseModel):
    id: int = Field(..., alias="id")
    login: str = Field(..., alias="login")
    name: str | None = Field(default=None, alias="name")
    company: str | None = Field(default=None, alias="company")
    url: str = Field(..., alias="html_url")


class Label(BaseModel):
    id: int = Field(..., alias="id")
    name: str = Field(..., alias="name")
    description: str | None = Field(default=None, alias="description")


class Issue(BaseModel):
    id: int = Field(..., alias="id")
    number: int = Field(..., alias="number")
    url: str = Field(..., alias="html_url")
    title: str = Field(..., alias="title")
    state: str = Field(..., alias="state")
    labels: list[Label] = Field(..., alias="labels")
    when_created: str | None = Field(default=None, alias="created_at")
    when_closed: str | None = Field(default=None, alias="closed_at")


class IssueDetail(Issue):
    body: str | None = Field(default=None, alias="body")


# endregion


class GitHubSettings(BaseModel):
    base_url: str = "https://api.github.com"
    token: str


class GitHubPlugin:
    def __init__(self, settings: GitHubSettings):
        self.settings = settings

    @kernel_function
    async def get_user_profile(self) -> "User":
        async with self.create_client() as client:
            response = await self.make_request(client, "/user")
            return User(**response)

    @kernel_function
    async def get_repository(self, organization: str, repo: str) -> "Repo":
        async with self.create_client() as client:
            response = await self.make_request(client, f"/repos/{organization}/{repo}")
            return Repo(**response)

    @kernel_function
    async def get_issues(
        self,
        organization: str,
        repo: str,
        max_results: int | None = None,
        state: str = "",
        label: str = "",
        assignee: str = "",
    ) -> list["Issue"]:
        async with self.create_client() as client:
            path = f"/repos/{organization}/{repo}/issues?"
            path = self.build_query(path, "state", state)
            path = self.build_query(path, "assignee", assignee)
            path = self.build_query(path, "labels", label)
            path = self.build_query(path, "per_page", str(max_results) if max_results else "")
            response = await self.make_request(client, path)
            return [Issue(**issue) for issue in response]
    
    @kernel_function
    async def get_commits(
        self,
        organization: str,
        repo: str,
        max_results: int | None = None,
        author: str = "",
        since: str = "",
        until: str = "",
    ) -> list[dict]:
        """
        Retrieve commits from a GitHub repository.

        Args:
            organization (str): The organization or user name.
            repo (str): The repository name.
            max_results (int, optional): Maximum number of commits to return.
            author (str, optional): Filter by commit author.
            since (str, optional): Only commits after this date (ISO 8601).
            until (str, optional): Only commits before this date (ISO 8601).

        Returns:
            list[dict]: A list of commit data dictionaries.
        """
        async with self.create_client() as client:
            path = f"/repos/{organization}/{repo}/commits?"
            path = self.build_query(path, "author", author)
            path = self.build_query(path, "since", since)
            path = self.build_query(path, "until", until)
            path = self.build_query(path, "per_page", str(max_results) if max_results else "")
            response = await self.make_request(client, path)
            return response
        
    @kernel_function
    async def get_commit_detail(
        self,
        organization: str,
        repo: str,
        commit_sha: str,
    ) -> dict:
        """
        Retrieve details for a specific commit in a GitHub repository.

        Args:
            organization (str): The organization or user name.
            repo (str): The repository name.
            commit_sha (str): The commit SHA.

        Returns:
            dict: The commit details.
        """
        async with self.create_client() as client:
            path = f"/repos/{organization}/{repo}/commits/{commit_sha}"
            response = await self.make_request(client, path)
            return response
        
    @kernel_function
    async def get_commit_diff(
        self,
        organization: str,
        repo: str,
        base_commit: str,
        head_commit: str,
    ) -> dict:
        """
        Retrieve the diff (code changes) between two commits in a GitHub repository.

        Args:
            organization (str): The organization or user name.
            repo (str): The repository name.
            base_commit (str): The base commit SHA.
            head_commit (str): The head commit SHA.

        Returns:
            dict: The comparison result including files changed, commits, and diff stats.
        """
        async with self.create_client() as client:
            path = f"/repos/{organization}/{repo}/compare/{base_commit}...{head_commit}"
            response = await self.make_request(client, path)
            return response
        
    @kernel_function
    async def create_git_issue_with_labels(
        self,
        organization: str,
        repo: str,
        title: str,
        body: str ,
        labels: list[str]
    ) -> dict:
        """
        Create a new issue with dummy text and labels in the specified repository.

        Args:
            organization (str): The organization or user name.
            repo (str): The repository name.
            title (str, optional): The title of the issue.
            body (str, optional): The body content of the issue.
            labels (list[str], optional): List of labels to assign.

        Returns:
            dict: The created issue details.
        """
        async with self.create_client() as client:
            path = f"/repos/{organization}/{repo}/issues"
            payload = {"title": title, "body": body, "labels": labels}
            print(f"POST REQUEST: {path}\nPayload: {payload}")
            response = await client.post(path, json=payload)
            response.raise_for_status()
            return response.json()

    @kernel_function
    async def get_issue_detail(self, organization: str, repo: str, issue_id: int) -> "IssueDetail":
        async with self.create_client() as client:
            path = f"/repos/{organization}/{repo}/issues/{issue_id}"
            response = await self.make_request(client, path)
            return IssueDetail(**response)

    def create_client(self) -> httpx.AsyncClient:
        headers = {
            "User-Agent": "request",
            "Accept": "application/vnd.github+json",
            "Authorization": f"Bearer {self.settings.token}",
            "X-GitHub-Api-Version": "2022-11-28",
        }
        return httpx.AsyncClient(base_url=self.settings.base_url, headers=headers, timeout=5)

    @staticmethod
    def build_query(path: str, key: str, value: str) -> str:
        if value:
            return f"{path}{key}={value}&"
        return path

    @staticmethod
    async def make_request(client: httpx.AsyncClient, path: str) -> dict:
        print(f"REQUEST: {path}\n")
        response = await client.get(path)
        response.raise_for_status()
        return response.json()

## 6. Agent Definition

Finally we are ready to instantiate a ChatCompletionAgent with its Instructions, associated Kernel, and the default Arguments and Execution Settings. In this case, we desire to have the any plugin functions automatically executed.

>You will need to define settings for either OpenAI or Azure OpenAI and also for GitHub , before coming to this step.

In [21]:
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.functions.kernel_arguments import KernelArguments
from semantic_kernel.kernel import Kernel

# Import GitHubPlugin and GitHubSettings directly from the notebook
from __main__ import GitHubPlugin, GitHubSettings

inst_template = f"""You are an agent designed to query and retrieve information from a single GitHub repository in a read-only manner.
        You are also able to access the profile of the active user.
        Use the current date and time to provide up-to-date details or time-sensitive responses.
        Use the values from arguments:
        - The repository name: {{repo_name}}
        - The current datetime: {{now}}
"""
kernel = Kernel()
# Add the AzureChatCompletion AI Service to the Kernel
service_id = "serv-git-chat-1"
kernel.add_service(AzureChatCompletion(service_id=service_id))

settings = kernel.get_prompt_execution_settings_from_service_id(service_id=service_id)
# Configure the function choice behavior to auto invoke kernel functions
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# Set your GitHub Personal Access Token (PAT) value here
gh_settings = GitHubSettings(token=os.getenv("GITHUB_PAT"))

kernel.add_plugin(plugin=GitHubPlugin(gh_settings), plugin_name="GithubPlugin")

# Create the agent
agent = ChatCompletionAgent(
    kernel=kernel,
    name="GitAssistantAgent",
    instructions=inst_template,
    arguments=KernelArguments(settings=settings,repo_name="harsha3187/git-dev-agent"),
)

#### 6.1 All set! Now, let's test the agent with a simple query to get the user profile.

In [22]:
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from datetime import datetime

chat_history = ChatHistory()

user_input = "What is my username?"

chat_history.add_message(ChatMessageContent(role=AuthorRole.USER, content=user_input))

async for response in agent.invoke(messages=chat_history):
    print(response.content)

REQUEST: /user

Your GitHub username is harsha3187.


#### 6.2 Lets ask about the repo details

In [26]:
user_input = "Describe the repo microsoft/semantic-kernel"

chat_history.add_message(ChatMessageContent(role=AuthorRole.USER, content=user_input))

arguments = KernelArguments(now=datetime.now().strftime("%Y-%m-%d %H:%M"))

async for response in agent.invoke(messages=chat_history, arguments=arguments):
    print(f"{response.content}")

REQUEST: /repos/microsoft/semantic-kernel

The repository microsoft/semantic-kernel is designed to help developers integrate cutting-edge large language model (LLM) technology quickly and easily into their applications. You can find more information and access the repository here: https://github.com/microsoft/semantic-kernel.


#### 6.3 Describe the newest issue created in the repo.

In [ ]:
user_input = "Describe the newest issue created in the repo."

chat_history.add_message(ChatMessageContent(role=AuthorRole.USER, content=user_input))

arguments = KernelArguments(now=datetime.now().strftime("%Y-%m-%d %H:%M"))

async for response in agent.invoke(messages=chat_history, arguments=arguments):
    print(f"{response.content}")

#### 6.4 List the top 10 issues closed within the last week.

In [ ]:
user_input = "List the top 10 issues closed within the last week."

chat_history.add_message(ChatMessageContent(role=AuthorRole.USER, content=user_input))

arguments = KernelArguments(now=datetime.now().strftime("%Y-%m-%d %H:%M"))

async for response in agent.invoke(messages=chat_history, arguments=arguments):
    print(f"{response.content}")

#### And more examples...

In [ ]:
user_input = "How were these issues labeled?"

chat_history.add_message(ChatMessageContent(role=AuthorRole.USER, content=user_input))

arguments = KernelArguments(now=datetime.now().strftime("%Y-%m-%d %H:%M"))

async for response in agent.invoke(messages=chat_history, arguments=arguments):
    print(f"{response.content}")

In [ ]:
user_input = "List the 5 most recently opened issues with the 'Bug' label."

chat_history.add_message(ChatMessageContent(role=AuthorRole.USER, content=user_input))

arguments = KernelArguments(now=datetime.now().strftime("%Y-%m-%d %H:%M"))

async for response in agent.invoke(messages=chat_history, arguments=arguments):
    print(f"{response.content}")

In [ ]:
user_input = "What changed from Issue#412 to Issue#413?"

chat_history.add_message(ChatMessageContent(role=AuthorRole.USER, content=user_input))

arguments = KernelArguments(now=datetime.now().strftime("%Y-%m-%d %H:%M"))

async for response in agent.invoke(messages=chat_history, arguments=arguments):
    print(f"{response.content}")

In [ ]:
user_input = "Give me the last 5 commits with brief descriptions?"

chat_history.add_message(ChatMessageContent(role=AuthorRole.USER, content=user_input))

arguments = KernelArguments(now=datetime.now().strftime("%Y-%m-%d %H:%M"))

async for response in agent.invoke(messages=chat_history, arguments=arguments):
    print(f"{response.content}")

## 🔍 Additional Examples to Explore

Please try the following examples and add any relevant code snippets to the plugin functions as needed:

1. **Retrieve the list of contributors for a repository**  
   - Example: Fetch all contributors for the repository `microsoft/semantic-kernel`.

2. **Retrieve pull requests for a repository**  
   - Example: List all open pull requests for the repository `microsoft/semantic-kernel`.

3. **Analyze the most active contributors**  
   - Example: Identify contributors with the highest number of commits or pull requests.

4. **Who are the most active contributors in the repo microsoft/semantic-kernel?**  
   - Example: Analyze the activity of contributors in the `microsoft/semantic-kernel` repository to determine the most active ones.

## More resources

## 📚 Learning Materials for Semantic Kernel

### Official Documentation
- [Semantic Kernel Documentation](https://learn.microsoft.com/en-us/semantic-kernel/)
- [Semantic Kernel GitHub Repository](https://github.com/microsoft/semantic-kernel)

### Tutorials and Examples
- [Getting Started with Semantic Kernel](https://learn.microsoft.com/en-us/semantic-kernel/get-started/)
- [Example Chat Agent](https://learn.microsoft.com/en-us/semantic-kernel/frameworks/agent/examples/example-chat-agent?pivots=programming-language-python)
- [Semantic Kernel Python Samples](https://github.com/microsoft/semantic-kernel/tree/main/python/samples)

### Community and Support
- [Semantic Kernel Discussions](https://github.com/microsoft/semantic-kernel/discussions)
- [Stack Overflow - Semantic Kernel](https://stackoverflow.com/questions/tagged/semantic-kernel)